In [ ]:
!pip install pyspark
import pyspark
!wget https://tutorialmobile.000webhostapp.com/data.txt 
!wget https://tutorialmobile.000webhostapp.com/click_data_sample.csv
!wget https://tutorialmobile.000webhostapp.com/data-json.json

from pyspark import SparkContext, SparkConf
import collections
import re
from pyspark.sql import SQLContext
from pyspark.sql.types import Row
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import explode


def splitter(data):
  dataSplit= data.split()
  result = []
  for i in dataSplit:
    temp = re.sub('\W','',i)
    result.append(temp)
  return result
  
if __name__ == '__main__':
  conf = SparkConf().setMaster("local").setAppName("word couting")
  sc = SparkContext.getOrCreate(conf=conf)

  data = "/content/data.txt"
  file = open(data, "r") 
  DataFromFile = file.read()

  text = splitter(DataFromFile)
  rdd = sc.parallelize(text)
  #Spark RDD 
  mapWord = rdd.map(lambda word: (word.lower(),1))
  countWord = mapWord.reduceByKey(lambda x,y:x+y)
  print(countWord.collect())
  max = countWord.max(lambda x: x[1])
  print(max)

  #Spark DataFrame
  fields = [StructField("word", StringType(), True), StructField("time", IntegerType(), True)]
  schema = StructType(fields)
  sqlContext = SQLContext(sc)
  df_1 = sqlContext.createDataFrame(countWord, schema)
  print(df_1.count())
  print(df_1.printSchema())
  print(df_1.dtypes)
  print(df_1.show(5))

  df_2 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("/content/click_data_sample.csv")
  print(df_2.printSchema())
  print(df_2.show(5))

  df_3 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("/content/click_data_sample.csv")
  print(df_3.printSchema()) 

  df_4 = df_3.withColumnRenamed("click.at", "access_time").withColumnRenamed("user.id", "userID").withColumnRenamed("campaign.id", "campaignID")
  print(df_4.printSchema())

  df_json = sqlContext.read.option("multiline","true").json("/content/data-json.json")
  print(df_json.printSchema())
  print(df_json.show(5))
  print(df_json.first())
  student_data = df_json.select(explode(df_json.data).alias('student')).select('student.*').show(truncate=False)
  print(student_data)

  df_1.registerTempTable("df_1")
  print(sqlContext.sql(" SELECT * FROM df_1 where time >= 10").count())
  print(sqlContext.sql(" SELECT * FROM df_1 where time >= 10").show(5))


  print(df_1.filter(df_1["time"] > 10).count())
  print(df_1.filter(df_1["time"] > 10).show())
  print(df_1.select("word", "time").show())

--2021-02-21 17:54:51--  https://tutorialmobile.000webhostapp.com/data.txt
Resolving tutorialmobile.000webhostapp.com (tutorialmobile.000webhostapp.com)... 145.14.145.141, 2a02:4780:dead:2a12::1
Connecting to tutorialmobile.000webhostapp.com (tutorialmobile.000webhostapp.com)|145.14.145.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4109 (4.0K) [text/plain]
Saving to: ‘data.txt.5’

data.txt.5          100%[===================>]   4.01K  --.-KB/s    in 0s      

2021-02-21 17:54:51 (61.3 MB/s) - ‘data.txt.5’ saved [4109/4109]

--2021-02-21 17:54:51--  https://tutorialmobile.000webhostapp.com/click_data_sample.csv
Resolving tutorialmobile.000webhostapp.com (tutorialmobile.000webhostapp.com)... 145.14.145.141, 2a02:4780:dead:2a12::1
Connecting to tutorialmobile.000webhostapp.com (tutorialmobile.000webhostapp.com)|145.14.145.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14191872 (14M) [text/csv]
Saving to: ‘click_data_sample.cs

# Mục mới